# Conjugate Heat Transfer Example

This notebook demonstrates conjugate heat transfer simulations in PHASTA, including fluid-solid interactions and heat transfer across interfaces.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from phasta import FlowSolver, FlowConfig, Mesh, ConjugateHeatTransferConfig

# Set up plotting style
plt.style.use('seaborn-v0_8-whitegrid')

## Problem Setup

We'll simulate heat transfer in a heat exchanger with the following parameters:
- Channel dimensions: 0.1m × 0.02m × 0.1m
- Fin dimensions: 0.01m × 0.01m × 0.1m
- Hot fluid inlet temperature: 353K
- Cold fluid inlet temperature: 293K
- Hot fluid velocity: 0.1 m/s
- Cold fluid velocity: 0.1 m/s

In [ ]:
# Create base configuration
config = FlowConfig()
config.domain = {
    'width': 0.1,
    'height': 0.02,
    'depth': 0.1,
    'mesh_size': 0.001
}

config.flow = {
    'time_step': 0.01,
    'max_time': 100.0
}

config.boundary_conditions = {
    'hot_inlet': 'velocity',
    'hot_outlet': 'outflow',
    'cold_inlet': 'velocity',
    'cold_outlet': 'outflow',
    'walls': 'no-slip'
}

## Heat Exchanger Simulation

Let's simulate heat transfer in a finned heat exchanger.

In [ ]:
# Create mesh
mesh = Mesh.generate_heat_exchanger_3d(
    width=config.domain['width'],
    height=config.domain['height'],
    depth=config.domain['depth'],
    fin_width=0.01,
    fin_height=0.01,
    nx=100,
    ny=20,
    nz=100
)

# Plot mesh
plt.figure(figsize=(12, 4))
mesh.plot_slice('y', 0.5)
plt.title('Heat Exchanger Mesh (y-slice)')
plt.show()

In [ ]:
# Configure conjugate heat transfer
cht_config = ConjugateHeatTransferConfig()
cht_config.fluid_properties = {
    'hot_fluid': {
        'density': 1000.0,
        'viscosity': 0.001,
        'thermal_conductivity': 0.6,
        'specific_heat': 4200.0
    },
    'cold_fluid': {
        'density': 1000.0,
        'viscosity': 0.001,
        'thermal_conductivity': 0.6,
        'specific_heat': 4200.0
    }
}
cht_config.solid_properties = {
    'density': 2700.0,
    'thermal_conductivity': 237.0,
    'specific_heat': 900.0
}
cht_config.boundary_conditions = {
    'hot_inlet': {'type': 'temperature', 'value': 353.0},
    'cold_inlet': {'type': 'temperature', 'value': 293.0},
    'hot_outlet': {'type': 'outflow'},
    'cold_outlet': {'type': 'outflow'},
    'outer_walls': {'type': 'adiabatic'}
}
cht_config.interface = {
    'method': 'coupled',
    'tolerance': 1e-6,
    'max_iterations': 100
}

# Create and run solver
solver = FlowSolver(config, mesh, conjugate_heat_transfer_config=cht_config)
results = solver.solve()

print("\nHeat Exchanger Results:")
print(f"Simulation completed: {results.completed}")
print(f"Final time: {results.final_time}")
print(f"Number of time steps: {results.time_steps}")

## Visualize Results

Plot temperature fields, heat flux, and performance metrics.

In [ ]:
# Plot temperature fields
plt.figure(figsize=(15, 5))

# Plot hot fluid temperature
plt.subplot(131)
results.plot_fluid_temperature('hot')
plt.title('Hot Fluid Temperature')
plt.colorbar(label='Temperature (K)')

# Plot cold fluid temperature
plt.subplot(132)
results.plot_fluid_temperature('cold')
plt.title('Cold Fluid Temperature')
plt.colorbar(label='Temperature (K)')

# Plot solid temperature
plt.subplot(133)
results.plot_solid_temperature()
plt.title('Solid Temperature')
plt.colorbar(label='Temperature (K)')

plt.tight_layout()
plt.show()

In [ ]:
# Plot heat transfer metrics
plt.figure(figsize=(15, 5))

# Plot heat flux
plt.subplot(131)
results.plot_heat_flux()
plt.title('Heat Flux')
plt.colorbar(label='Heat Flux (W/m²)')

# Plot Nusselt number
plt.subplot(132)
results.plot_nusselt_number()
plt.title('Nusselt Number')
plt.colorbar(label='Nu')

# Plot effectiveness
plt.subplot(133)
results.plot_effectiveness()
plt.title('Heat Exchanger Effectiveness')
plt.colorbar(label='Effectiveness')

plt.tight_layout()
plt.show()

## Performance Analysis

Analyze heat exchanger performance and efficiency.

In [ ]:
# Plot performance metrics
plt.figure(figsize=(15, 5))

# Plot temperature profiles
plt.subplot(131)
results.plot_temperature_profiles()
plt.title('Temperature Profiles')
plt.xlabel('Position (m)')
plt.ylabel('Temperature (K)')
plt.grid(True)

# Plot heat transfer rate
plt.subplot(132)
results.plot_heat_transfer_rate()
plt.title('Heat Transfer Rate')
plt.xlabel('Time (s)')
plt.ylabel('Heat Transfer (W)')
plt.grid(True)

# Plot pressure drop
plt.subplot(133)
results.plot_pressure_drop()
plt.title('Pressure Drop')
plt.xlabel('Time (s)')
plt.ylabel('Pressure Drop (Pa)')
plt.grid(True)

plt.tight_layout()
plt.show()

## Parametric Study

Study the effect of different parameters on heat exchanger performance.

In [ ]:
# Plot parametric study results
plt.figure(figsize=(15, 5))

# Plot effect of fin spacing
plt.subplot(131)
results.plot_parametric_study('fin_spacing')
plt.title('Effect of Fin Spacing')
plt.xlabel('Fin Spacing (m)')
plt.ylabel('Effectiveness')
plt.grid(True)

# Plot effect of flow rate
plt.subplot(132)
results.plot_parametric_study('flow_rate')
plt.title('Effect of Flow Rate')
plt.xlabel('Flow Rate (m³/s)')
plt.ylabel('Effectiveness')
plt.grid(True)

# Plot effect of inlet temperature
plt.subplot(133)
results.plot_parametric_study('inlet_temperature')
plt.title('Effect of Inlet Temperature')
plt.xlabel('Temperature Difference (K)')
plt.ylabel('Effectiveness')
plt.grid(True)

plt.tight_layout()
plt.show()

## Exercises

1. Try different fin geometries
2. Experiment with different materials
3. Study the effect of flow arrangement
4. Compare different interface coupling methods

## Next Steps

- Try the multi-phase flow example
- Explore chemical reactions
- Learn about turbulent flows